## CIFAR-10 이미지 추출
### 클라이언트별 IID

In [1]:
import os
import random

import tensorflow as tf
import numpy as np
import pandas as pd
import PIL.Image as Image

2022-05-04 11:54:39.482321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 11:54:39.482340: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))

In [3]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

['T-shirt',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [4]:
clients = 10
div = clients + 1

In [5]:
counter = {}
output = os.path.abspath(os.path.expanduser('dataset'))
for idx, data in enumerate(zip(images, labels), start=0):
    image = Image.fromarray(data[0])
    label = labelnames[data[1]]
    num = counter.get(label, 0)
    party = num%div
    if party != 0:
        party = random.randint(1, clients)
    odir = os.path.join(output, f'{party}', label)
    os.makedirs(odir, exist_ok=True)
    opath = os.path.join(odir, f'{num:04d}.jpg')
    image.save(opath)
    counter[label] = num + 1

## 클라이언트별 학습

In [6]:
dataset_root = os.path.abspath(os.path.expanduser('dataset'))
with os.scandir(dataset_root) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            print(entry, entry.path, entry.name)

<DirEntry '2'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/2 2
<DirEntry '9'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/9 9
<DirEntry '3'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/3 3
<DirEntry '7'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/7 7
<DirEntry '10'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/10 10
<DirEntry '6'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/6 6
<DirEntry '1'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/1 1
<DirEntry '4'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/4 4
<DirEntry '8'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/8 8
<DirEntry '5'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFashion/EvenOdd/dataset/5 5
<DirEntry '0'> /home/harny/Github/flhub-experiments/KCC2022/MNISTFa

In [7]:
# Datasets
datasets = {}
dataset_root = os.path.abspath(os.path.expanduser('dataset'))
with os.scandir(dataset_root) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            if entry.name == '0':
                continue
            image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, validation_split=0.2)
            train = image_generator.flow_from_directory(entry.path,
                                                        classes=labelnames,
                                                        target_size=(32, 32),
                                                        subset='training',
                                                        shuffle=True)
            test = image_generator.flow_from_directory(entry.path,
                                                       classes=labelnames,
                                                       target_size=(32, 32),
                                                       subset='validation',
                                                       shuffle=True)
            datasets[entry.name] = (train, test)
datasets.keys()

Found 5123 images belonging to 10 classes.
Found 1275 images belonging to 10 classes.
Found 5013 images belonging to 10 classes.
Found 1247 images belonging to 10 classes.
Found 5070 images belonging to 10 classes.
Found 1262 images belonging to 10 classes.
Found 5099 images belonging to 10 classes.
Found 1269 images belonging to 10 classes.
Found 5073 images belonging to 10 classes.
Found 1264 images belonging to 10 classes.
Found 5177 images belonging to 10 classes.
Found 1289 images belonging to 10 classes.
Found 5079 images belonging to 10 classes.
Found 1264 images belonging to 10 classes.
Found 5062 images belonging to 10 classes.
Found 1262 images belonging to 10 classes.
Found 5156 images belonging to 10 classes.
Found 1283 images belonging to 10 classes.
Found 5093 images belonging to 10 classes.
Found 1270 images belonging to 10 classes.


dict_keys(['2', '9', '3', '7', '10', '6', '1', '4', '8', '5'])

In [8]:
# Datasets
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
datasets['0'] = image_generator.flow_from_directory(os.path.join(dataset_root, '0'),
                                                    classes=labelnames, 
                                                    target_size=(32, 32), 
                                                    shuffle=True)
datasets.keys()

Found 6370 images belonging to 10 classes.


dict_keys(['2', '9', '3', '7', '10', '6', '1', '4', '8', '5', '0'])

In [9]:
model = tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(64, activation='relu'),
                       tf.keras.layers.Dense(len(labelnames))])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

2022-05-04 11:54:47.033749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-04 11:54:47.033776: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-04 11:54:47.033788: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LuHa-X1C9): /proc/driver/nvidia/version does not exist
2022-05-04 11:54:47.033941: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# Base model
odir = os.path.join('models', '0', '0')
model.save(odir)

2022-05-04 11:54:47.265362: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/0/0/assets


In [11]:
rounds = 10
epochs = 10

In [20]:

for r in range(rounds):
    part = 0
    weights = []
    rpath = os.path.join('models', f'{r}')
    for c in range(1, clients+1):
        if r%2 != c%2:
            continue
        mpath = os.path.join(rpath, '0')
        model = tf.keras.models.load_model(mpath)
        history = model.fit(datasets[f'{c}'][0], epochs=epochs, verbose=0)
        opath = os.path.join(rpath, f'{c}')
        model.save(opath)
        if len(weights) == 0:
            weights = model.get_weights()
            part = part + 1
        else:
            for idx, weight in enumerate(model.get_weights()):
                weights[idx] = weights[idx] + weight
            part = part + 1
        print(f'Local train: round #{r} with clinent #{c}')
    for idx, weight in enumerate(weights):
        weights[idx] = weights[idx] / part
    mpath = os.path.join(rpath, '0')
    model = tf.keras.models.load_model(mpath)
    model.set_weights(weights)
    ndir = os.path.join('models', f'{r+1}')
    npath = os.path.join(ndir, '0')
    model.save(npath)
    metric = model.evaluate(datasets['0'], verbose=0)
    print(f'Global aggregation: round #{r+1} for {metric}')

INFO:tensorflow:Assets written to: models/0/2/assets
Local train: round #0 with clinent #2
INFO:tensorflow:Assets written to: models/0/4/assets
Local train: round #0 with clinent #4
INFO:tensorflow:Assets written to: models/0/6/assets
Local train: round #0 with clinent #6
INFO:tensorflow:Assets written to: models/0/8/assets
Local train: round #0 with clinent #8
INFO:tensorflow:Assets written to: models/0/10/assets
Local train: round #0 with clinent #10
INFO:tensorflow:Assets written to: models/1/0/assets
Global aggregation: round #1 for [0.5130490064620972, 0.8401883840560913]
INFO:tensorflow:Assets written to: models/1/1/assets
Local train: round #1 with clinent #1
INFO:tensorflow:Assets written to: models/1/3/assets
Local train: round #1 with clinent #3
INFO:tensorflow:Assets written to: models/1/5/assets
Local train: round #1 with clinent #5
INFO:tensorflow:Assets written to: models/1/7/assets
Local train: round #1 with clinent #7
INFO:tensorflow:Assets written to: models/1/9/assets

In [24]:
global_acc = []
global_loss = []
federated_acc = []
federated_loss = []
for r in range(1, rounds):
    part = 0
    weights = []
    mpath = os.path.join('models', f'{r}', '0')
    model = tf.keras.models.load_model(mpath)
    acc = []
    loss = []
    for c in range(0, clients+1):
        if c == 0:
            metric = model.evaluate(datasets[f'{c}'], verbose=0)
            global_loss.append(metric[0])
            global_acc.append(metric[1])
            print(f'round #{r} - Global: {metric}')
        else:
            if r%2 == c%2:
                continue
            metric = model.evaluate(datasets[f'{c}'][0], verbose=0)
            loss.append(metric[0])
            acc.append(metric[1])
    loss = sum(loss) / len(loss)
    acc = sum(acc) / len(acc)
    federated_loss.append(loss)
    federated_acc.append(acc)
    print(f'round #{r} - Federated: {(loss, acc)}')

round #1 - Global: [0.5130488872528076, 0.8401883840560913]
round #1 - Federated: (0.48167566657066346, 0.8530444502830505)
round #2 - Global: [0.380380243062973, 0.8817896246910095]
round #2 - Federated: (0.2848794341087341, 0.9000699520111084)
round #3 - Global: [0.39245352149009705, 0.8883830308914185]
round #3 - Federated: (0.24798900187015532, 0.9181190013885498)
round #4 - Global: [0.423841655254364, 0.8929356336593628]
round #4 - Federated: (0.2225220739841461, 0.9263599634170532)
round #5 - Global: [0.47575488686561584, 0.8949764370918274]
round #5 - Federated: (0.21194939017295839, 0.9363281965255738)
round #6 - Global: [0.48393383622169495, 0.8952904343605042]
round #6 - Federated: (0.193728768825531, 0.9390581965446472)
round #7 - Global: [0.4798671007156372, 0.8979591727256775]
round #7 - Federated: (0.17403753995895385, 0.945078432559967)
round #8 - Global: [0.5189034938812256, 0.9003139734268188]
round #8 - Federated: (0.15835869908332825, 0.9507721304893494)
round #9 - G

In [ ]:
global_acc = []
global_loss = []
federated_acc = []
federated_loss = []
for r in range(1, rounds):
    mpath = os.path.join('models', f'{r}', '0')
    model = tf.keras.models.load_model(mpath)
    acc = []
    loss = []
    for c in range(0, clients+1):
        if c == 0:
            metric = model.evaluate(datasets[f'{c}'], verbose=0)
            global_loss.append(metric[0])
            global_acc.append(metric[1])
            print(f'round #{r} - Global: {metric}')
        else:
            weights = []
            for weight in model.get_weights():
                weights.append(weight*clients)
            lpath = os.path.join('models', f'{r-1}', f'{c}')
            lmodel = tf.keras.models.load_model(lpath)
            for idx, weight in enumerate(lmodel.get_weights()):
                weights[idx] = (weights[idx]-weight)/(clients-1)
            model.set_weights(weights)
            metric = model.evaluate(datasets[f'{c}'], verbose=0)
            loss.append(metric[0])
            acc.append(metric[1])
    loss = sum(loss) / clients
    acc = sum(acc) / clients
    federated_loss.append(loss)
    federated_acc.append(acc)
    print(f'round #{r} - Federated: {(loss, acc)}')